<a href="https://colab.research.google.com/github/Babitha23/Cyberbullying-detection/blob/main/Bert%20word%20embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
!pip install wordcloud
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from google.colab import drive
%matplotlib inline
import re
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
from google.colab import drive
# Connect to google drive
drive.mount('/content/gdrive')

# Load data and set labels
data = pd.read_csv('gdrive/My Drive/Dissertation/final.csv', index_col='_unit_id')

Mounted at /content/gdrive


In [ ]:
sample = data.iloc[0:5,:]

In [ ]:
def cleancomments(text):
    tt = re.sub('\(created.at:\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d\)', '', str(text)) #replacing comment time
    tt1 = re.sub('<font color\=\"\#0066CC\"\>.*?\<\/font>', '', str(tt)) # removing html tags with username
    tt2 = re.sub('@[^\s]+', '', str(tt1)) #removing user tag
    #Remove unicode strings like "\u002c" and "x96"
    tmp = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', tt2) 
    temp = re.sub(r'[^\x00-\x7f]',r'',tmp)
    #removing url
    temp1 = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',temp)
    temp2 = re.sub(r'#([^\s]+)', r'\1', temp1) # removing hashtags
    #removing emoticons
    temp3 = re.sub(':\)|;\)|:-\)|<3|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', temp2)
    temp4 = ''.join([i for i in temp3 if not i.isdigit()]) #removing numbers
    temp5 = re.sub(r"( ?\.)\1+", '.', temp4) #replacing multiple . with single .
    temp6 = re.sub(r"( ?\!)+", '!', temp5) #replacing multiple ! with single !
    temp7 = re.sub(r"( ?\?)+", '?', temp6) #replacing multiple ? with single ?
    #temp8 = re.sub(r"(\_)+", '', temp7)
    temp8 = ' '.join( [w for w in temp7.split() if len(w)>1]) #removing single characters
    temp9 = re.sub(' +', ' ', temp8) #removing additional whitespaces
    return temp9

In [ ]:
for i in range(len(sample)):
  sample['owner_cmnt'] = data['owner_cmnt'].apply(lambda x: cleancomments(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sample.columns

Index(['question1', 'question2', 'clmn1', 'clmn10', 'clmn100', 'clmn101',
       'clmn102', 'clmn103', 'clmn104', 'clmn105',
       ...
       'clmn95', 'clmn96', 'clmn97', 'clmn98', 'clmn99', 'likes', 'owner_cmnt',
       'shared media', 'followed_by', 'follows'],
      dtype='object', length=202)

In [ ]:
sample = sample.drop(['likes','shared media', 'followed_by', 'follows'], axis=1)

In [ ]:
def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(str(word)):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

In [ ]:
import string
def removepunct(txt):
  remove = string.punctuation
  remove = remove.replace(".", "") # don't remove dot
  remove = remove.replace("!", "") # don't remove !
  remove = remove.replace("?", "") # don't remove ?
  pattern = r"[{}]".format(remove) # create the pattern
  text = re.sub(pattern, "", txt)
  return text

In [ ]:
for col in sample.columns[2:]:
  for i in range(len(sample)):
    sample[col] = sample[col].apply(lambda x: removepunct(str(x)))

In [ ]:
# for col in sample.columns[2:]:
#   for i in range(len(sample)):
#     sample[col] = sample[col].apply(lambda x: replaceElongated(str.split(x)))

In [ ]:
sample.loc[702714442,'clmn94']

'Gemma sua lindaaaaa'

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# for col in range(2,197):
#   for i in range(5):
#     if sample.iloc[i,col] is not np.NaN:
#       sent_l = sample.iloc[i,col].split()
#       for word in sent_l:
#         sample_df = sample.apply(lambda x: x.replace(regex=word, value=replaceElongated(word)))

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True) # Whether the model returns all hidden-states.
# Put the model in "evaluation" mode,meaning feed-forward operation.
model.eval()

In [ ]:
def tokenize(sentence, seg_id):
  # Add the special tokens.
  marked_text = "[CLS] " + sentence + " [SEP]"
  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)
  # Map the token strings to their vocabulary indices.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Mark each of the tokens as belonging to sentence "seg_id".
  segments_ids = [seg_id] * len(tokenized_text)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  return {'tokens_tensor':tokens_tensor, 'segments_tensors': segments_tensors}

In [ ]:
#copying only the structure of sample dataframe
sample_c = pd.DataFrame().reindex_like(sample)

In [99]:
s = sample.replace('nan', 'a')
sentences = s.drop(['question1','question2'], axis=1).to_numpy()
sentences[0][1]

'larry'

In [ ]:
def embed(sentence):
  a=0
  tempdict = tokenize(sentence, a)
  with torch.no_grad():
    outputs = model(tempdict['tokens_tensor'], tempdict['segments_tensors'])
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
  a+=1
  return token_embeddings

In [112]:
sample.columns[3:]

Index(['clmn1', 'clmn10', 'clmn100', 'clmn101', 'clmn102', 'clmn103',
       'clmn104', 'clmn105', 'clmn106', 'clmn107',
       ...
       'clmn91', 'clmn92', 'clmn93', 'clmn94', 'clmn95', 'clmn96', 'clmn97',
       'clmn98', 'clmn99', 'owner_cmnt'],
      dtype='object', length=196)

In [113]:
# embedding = {}
# for i in range(len(sentences)):
#   for j in range(len(sentences[i])):
#     embedding[sentences[i][j]] = embed(sentences[i][j])

for col in sample.columns[2:]:
  sample[col] = sample[col].apply(lambda x: embed(x))

In [115]:
sample.iloc[0,2].size() #[number of tokens, 13 layers, 768 features]

torch.Size([6, 13, 768])

In [123]:
sample.sample(2)

,question1,question2,clmn1,clmn10,clmn100,clmn101,clmn102,clmn103,clmn104,clmn105,...,clmn91,clmn92,clmn93,clmn94,clmn95,clmn96,clmn97,clmn98,clmn99,owner_cmnt
_unit_id,,,,,,,,,,,,,,,,,,,,,
702714444,noneAgg,noneBll,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....",...,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0...."
702714442,noneAgg,noneBll,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....",...,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0...."


In [124]:
# Stores the token vectors, with shape [<num of tokens> x 768]
#token_vecs_sum = []

# each `embedding.values` is a [<num of tokens> x 12 x 768] tensor.

# For each token in the sentence...

def vec(embedd):
  token_vecs_sum = []
  for token in embedd:
    # `token` is a [12 x 768] tensor
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)
  return token_vecs_sum

for col in sample.columns[2:]:
  sample[col] = sample[col].apply(lambda x: vec(x))

In [130]:
print("Number of tokens:", len(sample.iloc[0,2]))
print("Number of features:", sample.iloc[0,2][0].size())

Number of tokens: 6
Number of features: torch.Size([768])


In [ ]:
max()

In [ ]:
count = 0
for i in range(5):
  count = count + len(sentences[i])

print("total number of sentences including NA:", count) #196 columns * 5 rows = 980

total number of sentences including NA: 980


In [68]:
sample_c = sample.copy(deep=True)

In [95]:
token_vec[sample.iloc[0,3]][0].size()

torch.Size([768])

In [82]:
type(sample.iloc[0,4])

str

AttributeError: ignored

In [ ]:
# count1 = 0
# for i in range(5):
#   #count1 += np.count_nonzero(sentences[i] == 'a')
#   count1 = count1 + (sentences[i] != 'a').sum()
# print("Senternces without NA:", count1)

In [ ]:
len(token_vec) #number of sentences that are not na

#repeated sentences occur once

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(num_channels, 
        implementation = 2, recurrent_activation = 'sigmoid'),
        input_shape=(input_length, input_dim)))

In [ ]:
# conditions = [
#     df['gender'].eq('male') & df['pet1'].eq(df['pet2']),
#     df['gender'].eq('female') & df['pet1'].isin(['cat', 'dog'])
# ]

# choices = [5,5]

# df['points'] = np.select(conditions, choices, default=0)

In [ ]:
# s = sample.replace(np.NaN, '')
# n = s.drop(['question1','question2'], axis=1).to_numpy()
# n

In [ ]:
## create label and sentence list
# sentences = n

In [ ]:
# ## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
# input_ids = [tokenizer.encode(sent, add_special_tokens=True,padding==True) for sent in sentences]
# labels = s.question1.values

# print("Actual sentence before tokenization: ",sentences[2])
# print("Encoded Input from dataset: ",input_ids[2])

# ## Create attention mask
# attention_masks = []
# ## Create a mask of 1 for all input tokens and 0 for all padding tokens
# attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
# print(attention_masks[2])

In [ ]:
# newsent = np.array([])
# for i in range(5):
#   newsent[i] = np.delete(sentences[i], np.where(sentences[i] == 'a'))

In [ ]:
# token_vec = {}
# for text, embedd in embedding.items():
#   token_vecs_sum = []
#   for token in embedd:
#     # `token` is a [12 x 768] tensor
#     # Sum the vectors from the last four layers.
#     sum_vec = torch.sum(token[-4:], dim=0)
#     # Use `sum_vec` to represent `token`.
#     token_vecs_sum.append(sum_vec)
#     token_vec[text] = token_vecs_sum

#print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))